In [1]:
# ! pip install -r ../../requirements.txt

In [2]:
import dotenv

dotenv.load_dotenv('./env.txt')

True

In [3]:
from langfuse import Langfuse
 
langfuse = Langfuse()
langfuse_prompt = langfuse.get_prompt("langchain-rag-prompt", cache_ttl_seconds=0)
print(langfuse_prompt.prompt)

Answer this question using the provided context and say thanks for contacting at the end.

{{question}}

Context:
{{context}}


In [4]:
online_prompt = langfuse_prompt.get_langchain_prompt()
online_prompt

'Answer this question using the provided context and say thanks for contacting at the end.\n\n{question}\n\nContext:\n{context}'

In [5]:
langfuse_prompt.config

{'model': 'gpt-3.5-turbo', 'temperature': 0}

In [6]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Artificial intelligence", load_max_docs=2, doc_content_chars_max=1000).load()
docs

[Document(metadata={'title': 'Artificial intelligence', 'summary': 'Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.\nSome high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general 

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
documents=text_splitter.split_documents(docs)
for doc in documents[:5]:
    print(doc.page_content, "\n")
print(len(documents))

Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence 

use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs. 

Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools 

generative and creative tools (e.g., ChatGPT, Apple Intelligence, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general app

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [19]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2},
)

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=langfuse_prompt.config["model"], temperature=langfuse_prompt.config["temperature"])

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(online_prompt)

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [17]:
from langfuse.callback import CallbackHandler
from langfuse.decorators import langfuse_context

langfuse_handler = CallbackHandler(
    session_id="langchain-test",
    user_id="kartik@gmail.com",
    metadata={"is_notebook": True}
)
# langfuse_handler = langfuse_context.get_current_langchain_handler()
print(rag_chain.invoke("what is AI ?", config={"callbacks": [langfuse_handler]}))

# optional
langfuse_handler.flush()

AI, or Artificial Intelligence, refers to intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that focuses on developing methods and software that enable machines to perceive their environment, learn, and make intelligent decisions to achieve defined goals. AI has various applications, such as advanced web search engines, recommendation systems, speech interaction, autonomous vehicles, generative tools, and superhuman play in strategy games. The field of AI has evolved over the years, with significant advancements in deep learning and the transformer architecture leading to an AI boom in the early 2020s.

Thanks for contacting!


# Model based Evaluation
- we can use available or a custom template
- can run evaluations based on filter criterias
- Finally we the scores are updated

In [20]:
langfuse_handler = CallbackHandler(
    session_id="model-based-eval-check",
    user_id="kartik@gmail.com",
    metadata={"is_notebook": True},
    tags = ["lang_check"]
)
rag_chain.invoke("cosa è AI ?", config={"callbacks": [langfuse_handler]})

'Artificial intelligence (AI) is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. AI has various applications such as advanced web search engines, recommendation systems, interacting via human speech, autonomous vehicles, generative and creative tools, and superhuman play in strategy games. AI was founded as an academic discipline in 1956 and has gone through cycles of optimism and disappointment. The growing use of AI in the 21st century is influencing societal and economic shifts towards increased automation and data-driven decision-making. The long-term goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics.\n\nThanks for co

# Datasets